# ML CUP 2022

## Regression based on k-nearest neighbors

This notebook creates a k-nearest neighbors for regression (KNR) model to generalize the problem of the ML cup 2022. It searches the best combination of hyperparameters performing a grid searches over a given range of values.

Hyperparameters considered for the grid search:

1. n_neighbors
2. weights
3. algorithm
4. metric

### Loading libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg as LA

from sklearn.metrics import make_scorer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

import joblib

import math

### Definition of the Mean Euclidean Distance

In [2]:
def my_mean_euclidean_distance(y_true, y_pred):
    points = len(y_true)
    tot_sum = 0
    for i in range (points):
        tot_sum += LA.norm(y_true[i] - y_pred[i])
    
    return tot_sum / points

In [3]:
mean_euclidean_distance = make_scorer(my_mean_euclidean_distance, greater_is_better=False)

### Loading data

In [4]:
colnames = ['id', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'target1', 'target2']
mlcup_tr = pd.read_csv("./dataset/ml_cup22/ML-CUP22-TR.csv", sep = ",", names=colnames)
mlcup_tr = mlcup_tr.iloc[1:, :]
mlcup_tr = mlcup_tr.drop('id', axis=1)

In [5]:
x_mlcup_tr = mlcup_tr.iloc[:, 0:9].values
y_mlcup_tr = mlcup_tr.iloc[:, 9:11].values

We used the function below to normalize our training set (both input and target) according to a min-max normalization

In [6]:
x_cols = len(x_mlcup_tr[0])

max_col_value_x = [None]*x_cols
max_vl = None

min_col_value_x = [None]*x_cols
min_vl = None

for i in range(x_cols):
    col = x_mlcup_tr[:, i]
    max_vl = np.amax(col)
    min_vl = np.amin(col)
    
    x_mlcup_tr[:, i] = (x_mlcup_tr[:, i] - min_vl) / (max_vl - min_vl)
    
    max_col_value_x[i] = max_vl
    min_col_value_x[i] = min_vl
    
    
    
y_cols = len(y_mlcup_tr[0])

max_col_value_y = [None]*y_cols

min_col_value_y = [None]*y_cols

for i in range(y_cols):
    col = y_mlcup_tr[:, i]
    max_vl = np.amax(col)
    min_vl = np.amin(col)
    
    y_mlcup_tr[:, i] = (y_mlcup_tr[:, i] - min_vl) / (max_vl - min_vl)
    
    max_col_value_y[i] = max_vl
    min_col_value_y[i] = min_vl 

y1_mlcup_tr = y_mlcup_tr[:, 0]
y2_mlcup_tr = y_mlcup_tr[:, 1]

We create a validation set, splitting the training set in 70% and 30%. We do this in order not to fall in an overfitting case and to evaluate the error.

In [7]:
subset_size = int(0.3 * len(x_mlcup_tr))

index = np.random.choice(len(x_mlcup_tr), subset_size, replace=False)
index = np.sort(index)

x_val_set = x_mlcup_tr[index]
x_tr_set = np.delete(x_mlcup_tr, index, 0)

y_val_set = y_mlcup_tr[index]
y_tr_set = np.delete(y_mlcup_tr, index, 0)

### Grid search

In [8]:
n_neighbors = np.arange(5, 30)
weights = ['uniform', 'distance']
algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
metric = ['l1', 'l2', 'euclidean']

param_grid = dict(
    n_neighbors = n_neighbors,
    weights = weights,
    algorithm = algorithm,
    metric = metric
)

grid = GridSearchCV(
    KNeighborsRegressor(),
    param_grid = param_grid,
    cv = 5,
    scoring = mean_euclidean_distance,
    verbose = 4
)

grid.fit(x_tr_set, y_tr_set)

print(
    "The best parameters are %s with a score of %0.5f"
    % (grid.best_params_, grid.best_score_)
)

Fitting 5 folds for each of 600 candidates, totalling 3000 fits
[CV 1/5] END algorithm=auto, metric=l1, n_neighbors=5, weights=uniform;, score=-0.085 total time=   0.0s
[CV 2/5] END algorithm=auto, metric=l1, n_neighbors=5, weights=uniform;, score=-0.080 total time=   0.0s
[CV 3/5] END algorithm=auto, metric=l1, n_neighbors=5, weights=uniform;, score=-0.078 total time=   0.0s
[CV 4/5] END algorithm=auto, metric=l1, n_neighbors=5, weights=uniform;, score=-0.084 total time=   0.0s
[CV 5/5] END algorithm=auto, metric=l1, n_neighbors=5, weights=uniform;, score=-0.075 total time=   0.0s
[CV 1/5] END algorithm=auto, metric=l1, n_neighbors=5, weights=distance;, score=-0.086 total time=   0.0s
[CV 2/5] END algorithm=auto, metric=l1, n_neighbors=5, weights=distance;, score=-0.080 total time=   0.0s
[CV 3/5] END algorithm=auto, metric=l1, n_neighbors=5, weights=distance;, score=-0.078 total time=   0.0s
[CV 4/5] END algorithm=auto, metric=l1, n_neighbors=5, weights=distance;, score=-0.084 total 

[CV 3/5] END algorithm=auto, metric=l1, n_neighbors=12, weights=distance;, score=-0.073 total time=   0.0s
[CV 4/5] END algorithm=auto, metric=l1, n_neighbors=12, weights=distance;, score=-0.078 total time=   0.0s
[CV 5/5] END algorithm=auto, metric=l1, n_neighbors=12, weights=distance;, score=-0.065 total time=   0.0s
[CV 1/5] END algorithm=auto, metric=l1, n_neighbors=13, weights=uniform;, score=-0.082 total time=   0.0s
[CV 2/5] END algorithm=auto, metric=l1, n_neighbors=13, weights=uniform;, score=-0.079 total time=   0.0s
[CV 3/5] END algorithm=auto, metric=l1, n_neighbors=13, weights=uniform;, score=-0.074 total time=   0.0s
[CV 4/5] END algorithm=auto, metric=l1, n_neighbors=13, weights=uniform;, score=-0.079 total time=   0.0s
[CV 5/5] END algorithm=auto, metric=l1, n_neighbors=13, weights=uniform;, score=-0.065 total time=   0.0s
[CV 1/5] END algorithm=auto, metric=l1, n_neighbors=13, weights=distance;, score=-0.082 total time=   0.0s
[CV 2/5] END algorithm=auto, metric=l1, n_

[CV 2/5] END algorithm=auto, metric=l1, n_neighbors=24, weights=uniform;, score=-0.078 total time=   0.0s
[CV 3/5] END algorithm=auto, metric=l1, n_neighbors=24, weights=uniform;, score=-0.073 total time=   0.0s
[CV 4/5] END algorithm=auto, metric=l1, n_neighbors=24, weights=uniform;, score=-0.081 total time=   0.0s
[CV 5/5] END algorithm=auto, metric=l1, n_neighbors=24, weights=uniform;, score=-0.067 total time=   0.0s
[CV 1/5] END algorithm=auto, metric=l1, n_neighbors=24, weights=distance;, score=-0.080 total time=   0.0s
[CV 2/5] END algorithm=auto, metric=l1, n_neighbors=24, weights=distance;, score=-0.077 total time=   0.0s
[CV 3/5] END algorithm=auto, metric=l1, n_neighbors=24, weights=distance;, score=-0.073 total time=   0.0s
[CV 4/5] END algorithm=auto, metric=l1, n_neighbors=24, weights=distance;, score=-0.080 total time=   0.0s
[CV 5/5] END algorithm=auto, metric=l1, n_neighbors=24, weights=distance;, score=-0.067 total time=   0.0s
[CV 1/5] END algorithm=auto, metric=l1, n

[CV 2/5] END algorithm=auto, metric=l2, n_neighbors=11, weights=uniform;, score=-0.078 total time=   0.0s
[CV 3/5] END algorithm=auto, metric=l2, n_neighbors=11, weights=uniform;, score=-0.072 total time=   0.0s
[CV 4/5] END algorithm=auto, metric=l2, n_neighbors=11, weights=uniform;, score=-0.079 total time=   0.0s
[CV 5/5] END algorithm=auto, metric=l2, n_neighbors=11, weights=uniform;, score=-0.067 total time=   0.0s
[CV 1/5] END algorithm=auto, metric=l2, n_neighbors=11, weights=distance;, score=-0.079 total time=   0.0s
[CV 2/5] END algorithm=auto, metric=l2, n_neighbors=11, weights=distance;, score=-0.077 total time=   0.0s
[CV 3/5] END algorithm=auto, metric=l2, n_neighbors=11, weights=distance;, score=-0.072 total time=   0.0s
[CV 4/5] END algorithm=auto, metric=l2, n_neighbors=11, weights=distance;, score=-0.079 total time=   0.0s
[CV 5/5] END algorithm=auto, metric=l2, n_neighbors=11, weights=distance;, score=-0.067 total time=   0.0s
[CV 1/5] END algorithm=auto, metric=l2, n

[CV 4/5] END algorithm=auto, metric=l2, n_neighbors=20, weights=distance;, score=-0.080 total time=   0.0s
[CV 5/5] END algorithm=auto, metric=l2, n_neighbors=20, weights=distance;, score=-0.065 total time=   0.0s
[CV 1/5] END algorithm=auto, metric=l2, n_neighbors=21, weights=uniform;, score=-0.080 total time=   0.0s
[CV 2/5] END algorithm=auto, metric=l2, n_neighbors=21, weights=uniform;, score=-0.077 total time=   0.0s
[CV 3/5] END algorithm=auto, metric=l2, n_neighbors=21, weights=uniform;, score=-0.072 total time=   0.0s
[CV 4/5] END algorithm=auto, metric=l2, n_neighbors=21, weights=uniform;, score=-0.081 total time=   0.0s
[CV 5/5] END algorithm=auto, metric=l2, n_neighbors=21, weights=uniform;, score=-0.066 total time=   0.0s
[CV 1/5] END algorithm=auto, metric=l2, n_neighbors=21, weights=distance;, score=-0.080 total time=   0.0s
[CV 2/5] END algorithm=auto, metric=l2, n_neighbors=21, weights=distance;, score=-0.077 total time=   0.0s
[CV 3/5] END algorithm=auto, metric=l2, n_

...

[CV 1/5] END algorithm=brute, metric=l2, n_neighbors=20, weights=uniform;, score=-0.081 total time=   0.0s
[CV 2/5] END algorithm=brute, metric=l2, n_neighbors=20, weights=uniform;, score=-0.077 total time=   0.0s
[CV 3/5] END algorithm=brute, metric=l2, n_neighbors=20, weights=uniform;, score=-0.072 total time=   0.0s
[CV 4/5] END algorithm=brute, metric=l2, n_neighbors=20, weights=uniform;, score=-0.080 total time=   0.0s
[CV 5/5] END algorithm=brute, metric=l2, n_neighbors=20, weights=uniform;, score=-0.065 total time=   0.0s
[CV 1/5] END algorithm=brute, metric=l2, n_neighbors=20, weights=distance;, score=-0.080 total time=   0.0s
[CV 2/5] END algorithm=brute, metric=l2, n_neighbors=20, weights=distance;, score=-0.077 total time=   0.0s
[CV 3/5] END algorithm=brute, metric=l2, n_neighbors=20, weights=distance;, score=-0.072 total time=   0.0s
[CV 4/5] END algorithm=brute, metric=l2, n_neighbors=20, weights=distance;, score=-0.080 total time=   0.0s
[CV 5/5] END algorithm=brute, met

[CV 5/5] END algorithm=brute, metric=l2, n_neighbors=27, weights=distance;, score=-0.067 total time=   0.0s
[CV 1/5] END algorithm=brute, metric=l2, n_neighbors=28, weights=uniform;, score=-0.081 total time=   0.0s
[CV 2/5] END algorithm=brute, metric=l2, n_neighbors=28, weights=uniform;, score=-0.078 total time=   0.0s
[CV 3/5] END algorithm=brute, metric=l2, n_neighbors=28, weights=uniform;, score=-0.073 total time=   0.0s
[CV 4/5] END algorithm=brute, metric=l2, n_neighbors=28, weights=uniform;, score=-0.080 total time=   0.0s
[CV 5/5] END algorithm=brute, metric=l2, n_neighbors=28, weights=uniform;, score=-0.068 total time=   0.0s
[CV 1/5] END algorithm=brute, metric=l2, n_neighbors=28, weights=distance;, score=-0.081 total time=   0.0s
[CV 2/5] END algorithm=brute, metric=l2, n_neighbors=28, weights=distance;, score=-0.078 total time=   0.0s
[CV 3/5] END algorithm=brute, metric=l2, n_neighbors=28, weights=distance;, score=-0.073 total time=   0.0s
[CV 4/5] END algorithm=brute, met

[CV 5/5] END algorithm=brute, metric=euclidean, n_neighbors=10, weights=uniform;, score=-0.067 total time=   0.0s
[CV 1/5] END algorithm=brute, metric=euclidean, n_neighbors=10, weights=distance;, score=-0.080 total time=   0.0s
[CV 2/5] END algorithm=brute, metric=euclidean, n_neighbors=10, weights=distance;, score=-0.078 total time=   0.0s
[CV 3/5] END algorithm=brute, metric=euclidean, n_neighbors=10, weights=distance;, score=-0.072 total time=   0.0s
[CV 4/5] END algorithm=brute, metric=euclidean, n_neighbors=10, weights=distance;, score=-0.082 total time=   0.0s
[CV 5/5] END algorithm=brute, metric=euclidean, n_neighbors=10, weights=distance;, score=-0.067 total time=   0.0s
[CV 1/5] END algorithm=brute, metric=euclidean, n_neighbors=11, weights=uniform;, score=-0.079 total time=   0.0s
[CV 2/5] END algorithm=brute, metric=euclidean, n_neighbors=11, weights=uniform;, score=-0.078 total time=   0.0s
[CV 3/5] END algorithm=brute, metric=euclidean, n_neighbors=11, weights=uniform;, s

[CV 3/5] END algorithm=brute, metric=euclidean, n_neighbors=17, weights=distance;, score=-0.072 total time=   0.0s
[CV 4/5] END algorithm=brute, metric=euclidean, n_neighbors=17, weights=distance;, score=-0.080 total time=   0.0s
[CV 5/5] END algorithm=brute, metric=euclidean, n_neighbors=17, weights=distance;, score=-0.065 total time=   0.0s
[CV 1/5] END algorithm=brute, metric=euclidean, n_neighbors=18, weights=uniform;, score=-0.080 total time=   0.0s
[CV 2/5] END algorithm=brute, metric=euclidean, n_neighbors=18, weights=uniform;, score=-0.076 total time=   0.0s
[CV 3/5] END algorithm=brute, metric=euclidean, n_neighbors=18, weights=uniform;, score=-0.071 total time=   0.0s
[CV 4/5] END algorithm=brute, metric=euclidean, n_neighbors=18, weights=uniform;, score=-0.080 total time=   0.0s
[CV 5/5] END algorithm=brute, metric=euclidean, n_neighbors=18, weights=uniform;, score=-0.066 total time=   0.0s
[CV 1/5] END algorithm=brute, metric=euclidean, n_neighbors=18, weights=distance;, sc

[CV 3/5] END algorithm=brute, metric=euclidean, n_neighbors=25, weights=uniform;, score=-0.073 total time=   0.0s
[CV 4/5] END algorithm=brute, metric=euclidean, n_neighbors=25, weights=uniform;, score=-0.081 total time=   0.0s
[CV 5/5] END algorithm=brute, metric=euclidean, n_neighbors=25, weights=uniform;, score=-0.067 total time=   0.0s
[CV 1/5] END algorithm=brute, metric=euclidean, n_neighbors=25, weights=distance;, score=-0.081 total time=   0.0s
[CV 2/5] END algorithm=brute, metric=euclidean, n_neighbors=25, weights=distance;, score=-0.077 total time=   0.0s
[CV 3/5] END algorithm=brute, metric=euclidean, n_neighbors=25, weights=distance;, score=-0.073 total time=   0.0s
[CV 4/5] END algorithm=brute, metric=euclidean, n_neighbors=25, weights=distance;, score=-0.080 total time=   0.0s
[CV 5/5] END algorithm=brute, metric=euclidean, n_neighbors=25, weights=distance;, score=-0.067 total time=   0.0s
[CV 1/5] END algorithm=brute, metric=euclidean, n_neighbors=26, weights=uniform;, s

In [9]:
knr = grid.best_estimator_

In [10]:
pred_label_knr = knr.predict(x_val_set)

After we train the model, we denormalize the data and we evaluate the error.

In [11]:
#Denormalization
def deNormalizer(pred_labels, max_col_value_y, min_col_value_y):
    
    y_cols = len(pred_labels[0])
    
    for i in range(y_cols):
        
        pred_labels[:, i] = pred_labels[:, i] * (max_col_value_y[i] - min_col_value_y[i]) + min_col_value_y[i]
    
    return pred_labels

In [12]:
pred_label_knr = deNormalizer(pred_label_knr, max_col_value_y, min_col_value_y)
y_val_set = deNormalizer(y_val_set, max_col_value_y, min_col_value_y)

In [14]:
# Mean euclidean distance
points = y_val_set.shape[0]
tot_sum = 0
for i in range (points):
    tot_sum += math.sqrt(math.pow((y_val_set[i][0] - pred_label_knr[i][0]), 2)
                         + math.pow((y_val_set[i][1] - pred_label_knr[i][1]), 2))
    
print('MEE on the validation set:', tot_sum / points)

MEE on the validation set: 1.416060706554296
